Hey Guido,


here is a small example notebook that shows how I use the kdt search.

The input files are here:
https://www.dropbox.com/sh/wm272v5dqgbpu8k/AABKb356GCXPRUu3S0lehiuia/brep%20input?dl=0
You will need the files GoCcoordinates.dat and GCcoordinates.dat from this dropbox.
Please specify the path to where you have stored them in the next cell.

There are two more files that are needed, the parameters.hoc file and the BREP.py file, of which the paths also have to be specified in the next cell (if you use it from github, the paths should be fine)

I also attached a downsampling function in case you want to use a smaller dataset to start with!

Thanks a lot for looking into it, I am curious what comes out!

Best, Ines


In [23]:
# Input filenames and paths

%run ../BREPpy.py  
#the Parameters file
config_fn = '../input_files/Parameters.hoc'
#path to the coordinate files
p_co = './'


#filenames:
fi_go = p_co+'GoCcoordinates.dat'
fi_gr = p_co+'GCcoordinates.dat'

a = Pseudo_hoc()
a.convert_hoc_to_pickle(config_fn)



AA_GoC_con
AA_GoC_con
AAstep
AAstep
AAtoGoCzone
AAtoGoCzone
APCount
APCount
AlphaSynapse
AlphaSynapse
BBSaveState
BBSaveState
BC_ampa_decay
BC_ampa_decay
BC_ampa_rise
BC_ampa_rise
BC_gaba_decay
BC_gaba_decay
BC_gaba_rise
BC_gaba_rise
BCdensity
BCdensity
BCtoGoCratio
BCtoGoCratio
BCxrange
BCxrange
BCyrange
BCyrange
BCzone
BCzone
BCzrange
BCzrange
CV_gmax
CV_gmax
CVode
CVode
DEG
DEG
Deck
Deck
E
E
ENABLE_GRAPHICAL_INTERFACE
ENABLE_GRAPHICAL_INTERFACE
Exp2Syn
Exp2Syn
ExpSyn
ExpSyn
FARADAY
FARADAY
FInitializeHandler
FInitializeHandler
File
File
GAMMA
GAMMA
GCtoGoCratio
GCtoGoCratio
GCtoPCratio
GCtoPCratio
GCxrange
GCxrange
GCyrange
GCyrange
GCzrange
GCzrange
GLdepth
GLdepth
GUIMath
GUIMath
Gapcond
Gapcond
Glyph
Glyph
GoC_Ad_nseg
GoC_Ad_nseg
GoC_Ad_nsegpts
GoC_Ad_nsegpts
GoC_ApicalDendL
GoC_ApicalDendL
GoC_Atheta_max
GoC_Atheta_max
GoC_Atheta_min
GoC_Atheta_min
GoC_Atheta_stdev
GoC_Atheta_stdev
GoC_AxonL
GoC_AxonL
GoC_Axon_Xmax
GoC_Axon_Xmax
GoC_Axon_Xmin
GoC_Axon_Xmin
GoC_Axon_Ymax
GoC_Axon

In [20]:
! ls ..


AAtoGoCdistances.dat		GoCtoGoCsources.dat
AAtoGoCsegments.dat		GoCtoGoCtargets.dat
AAtoGoCsources.dat		Golgi cell checkout.ipynb
AAtoGoCtargets.dat		input_files
bprep2.py			Other_nogit
bprep3.py			output_brep
brep_cluster_serial.py		Output_comparison_evaluation.ipynb
brep_commented.scm		output_pybrep
brep-ivan			parallel_exp.py
BREPpy_first.ipynb		Parallelization experiments.ipynb
BREPpy.py			parallel_util.py
brep_try_on_cluster.py		par_d2.txt
brep.yy.scm			par_d.txt
cloudpickle			parser.scm
Cluster_scripts			PFtoGoCdistances.dat
compile_brep.sh			PFtoGoCsegments.dat
comp_ser_seq.py			PFtoGoCsources.dat
config_to_par.txt		PFtoGoCtargets.dat
Documentation			pseudo_hoc.pkl
example_simulation		pyBREP Description and to do list.ipynb
For_Guido			__pycache__
For_Sungho			README.md
Full_script_checkout.ipynb	requirements.txt
GCcoordinates.sorted.dat	run_pyBrep.py
GCTcoordinates.sorted.dat	saving_testing.py
GoCadendcoordinates.sorted.dat	sec_copy
GoCaxoncoordinates.dat		start_ipcluster.sh
GoC

AttributeError: 'str' object has no attribute 'open'

In [8]:
# Setup

#Read in config file
#h = neuron.hoc.HocObject()
#h = neuron.
neuron.h.xopen(config_fn)

# Set up the Golgi population, get dendrite points
gg = Golgi_pop(h)
gg.load_somata(fi_go)
gg.add_dendrites()

#Set up Granule population including aa and pf
gp = Granule_pop(h)
gp.load_somata(fi_gr)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()


In [13]:
# Access the different points:

# soma coordinates
golgi_som = gg.som
#gran_som = gp.som

# The dendrites in a ncells x npoints x dim structure
print (gg.a_dend.shape)
print (gg.b_dend.shape)

# All dendrite points mashed together 
# (qpts is some other class that stores which cell the points belonged to)
print (gg.qpts.coo.shape)

#The parallel fiber and ascending axon endpoints: (ncells x [begin, end] x dim)
print (gp.aa_dots.shape)
print (gp.pf_dots.shape)


(1995, 50, 3)
(1995, 24, 3)
(147630, 3)
(798000, 2, 3)
(798000, 2, 3)


In [15]:
c_rad = 15 #critical radius (distance around points
cc = Connect_2D(gg.qpts, gp.qpts_aa, c_rad) # This class is constructed on the query points
kdt, q_pts, lax_c, lax_range, lin_in_tree = cc.get_tree_setup() 
# the kdt is the 2D tree that contains the aa points, and q_pts contains the 2d projections of the Golgi query points

In [17]:
# query a single point in the 2D tree
def pt_in_tr (kdt, pt, c_rad):
    warnings.simplefilter('ignore')
    ind, = kdt.query_radius(pt, r = c_rad)
    return ind

#example:
a = pt_in_tr(kdt, q_pts[0], c_rad)
print (a)

[699964 620592 100583  53090 558203 242234 726650 283077 545714 601099
 642758 580733 552228 314127 358070 340115 784781 278942 552279 527459
 190550 697651 249989 262822 375618 241017 727845 483544 716643 746257
 429157 451788 507011 356132 509819 340864 705649  72728 337576 553758
    596 592145 450223 609067 101147 675497 181881  86406 529170 490368
 663466  81406  26668 755031 720898 415611 673926 437254 763130 593151
 454240 255336 254693 159257 476200 130492 155770 423228 645670 263469
 277898 617371 667910 654777 722026 491550  50343 756955 105881 478713
 296248 554513 435330 252858 414755 736488 293696  56534 350248 760294
 597238 742497 418875 546466 174609 566528 764993 703422 100912 543595
 506492 406414   2007 669132 100141 448018 274556 123615 407132 164247
  94681  65068 227504 300130 179910 439698 127210 372916 138279 574846
 512713 464833 357221  85411 190049 770600 398847 611099 186728 582317
 459852 204774 257430 745119 429158 599297 408855 177748 384360  42802
 18016

In [ ]:
#As the Connector object is initialized with objects from the Query class, here are the
#corresponding numpy arrays that contain the 3-dimensional points:
query_points = gg.qpts.coo
lin_tree_points = gp.qpts_aa.coo 

## Subsampling

In [ ]:
# subsampling (constant density, smaller patch)

p1 = p_co #path to coordinate files
p2 = p_co #path where the subsampled data goes


#output filenames
fi_go = p1+'GoCcoordinates.dat'
fi_gr = p1+'GCcoordinates.dat'
fo_go64 = p2+'GoCcoordinates_64.dat'
fo_gr64 = p2+'GCcoordinates_64.dat'
fo_go16 = p2+'GoCcoordinates_16.dat'
fo_gr16 = p2+'GCcoordinates_16.dat'
fo_go4 = p2+'GoCcoordinates_4.dat'
fo_gr4 = p2+'GCcoordinates_4.dat' 


# Original size of membrane patch:
x_r = np.array([0.0, 1500.0])
y_r = np.array([0.0, 750.0])
z_r = [0.0, 1000.0]
# Smaller sizes
rrs64 = [x_r/8, y_r/8, z_r]
rrs16 = [x_r/4, y_r/4, z_r]
rrs4 = [x_r/2, y_r/2, z_r]


def subsample_coords (rrs, fn_in, fn_out = 'input_files/downsampled.dat', save = True):
    res = []
    rnr = [0, 0]
    with open(fn_in, newline = '') as f, open (fn_out, 'w', newline = '') as w_f:
        rr = csv.reader(f, delimiter = ' ')
        if save: wr = csv.writer(w_f, delimiter = ' ')
        for line in rr:
            in_range = all([float(line[i])>rrs[i][0] and float(line[i])<rrs[i][1] for i in range(len(rrs))]) #check if in range
            if in_range: 
                if save: wr.writerow([float(line[j]) for j in range(len(rrs))])
                res.append([float(line[j]) for j in range(len(rrs))])
                rnr[0] = rnr[0]+1
            else:
                rnr[1] = rnr[1]+1
    print ('Subsampled {} of {}'.format(rnr[0], sum(rnr)))
    return res




_ = subsample_coords (rrs64, fi_go, fo_go64)
_ = subsample_coords (rrs64, fi_gr, fo_gr64)

'''_ = subsample_coords (rrs16, fi_go, fo_go16)
_ = subsample_coords (rrs16, fi_gr, fo_gr16)
_ = subsample_coords (rrs4, fi_go, fo_go4)
_ = subsample_coords (rrs4, fi_gr, fo_gr4)
'''